In [1]:
import os, sys 
%pwd

'/config/workspace/notebook'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str
    model_name: Path

In [4]:
from accident_severity.constants import *
from accident_severity.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            model_name=config.model_name
        )

        return model_evaluation_config 

In [6]:
import os, sys 
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from urllib.parse import urlparse
import json 
import numpy as np 
import joblib 
from accident_severity.constants import * 
from accident_severity.utils.common import read_yaml, create_directories, save_json

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config 

    def evaluation_metrics(self, actual, predicted):
        f1 = f1_score(actual, predicted, average="weighted")
        acc = accuracy_score(actual, predicted)

        return f1, acc

    def log_metrics(self, f1, acc):
        scores = {
            "f1_score": f1,
            "accuracy_score": acc
        }

        with open(self.config.metric_file_name, "w") as f:
            json.dump(scores, f)

    def evaluate_model(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]

        predictions = model.predict(X_test)

        score_f1, accuracy = self.evaluation_metrics(y_test, predictions)

        self.log_metrics(score_f1, accuracy)

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config/config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/model_evaluation]
